In [1]:
!pip install transformers peft --quiet

In [2]:
# 1 минута

!gdown 1iXzXQXyHQ-veqCQQbv6WH9vqgJWgpSGz
!gdown 1hWm6FEF0pMlz8PvTsiYemW3LEIhfBTqm
!gdown 1zsZWveqfa08nL-pOQJ2VfFSf5ucabk7k
!gdown 1Js_bwCWCb39-lFWVKzWqFSuq-RmdXeO_
!gdown 1HW3CUIqOPyXOet_LSwpUQvSN4cfCEf6k

Downloading...
From (original): https://drive.google.com/uc?id=1iXzXQXyHQ-veqCQQbv6WH9vqgJWgpSGz
From (redirected): https://drive.google.com/uc?id=1iXzXQXyHQ-veqCQQbv6WH9vqgJWgpSGz&confirm=t&uuid=6128a0a0-2c7a-4510-8c4b-26fb24c3bac1
To: /content/lora_bert_output_clothes.zip
100% 52.0M/52.0M [00:01<00:00, 41.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1hWm6FEF0pMlz8PvTsiYemW3LEIhfBTqm
From (redirected): https://drive.google.com/uc?id=1hWm6FEF0pMlz8PvTsiYemW3LEIhfBTqm&confirm=t&uuid=7699bf68-ddb6-4050-b560-70d33d30d47c
To: /content/lora_bert_output_shoes.zip
100% 50.7M/50.7M [00:01<00:00, 44.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1zsZWveqfa08nL-pOQJ2VfFSf5ucabk7k
From (redirected): https://drive.google.com/uc?id=1zsZWveqfa08nL-pOQJ2VfFSf5ucabk7k&confirm=t&uuid=827876bf-429b-4514-ba1f-c08d5f08eb01
To: /content/lora_bert_output_beauty.zip
100% 74.2M/74.2M [00:01<00:00, 46.4MB/s]
Downloading...
From (original): https://drive.google.co

In [3]:
!unzip lora_bert_output_clothes.zip -d lora_bert_output_clothes
!unzip lora_bert_output_shoes.zip   -d lora_bert_output_shoes
!unzip lora_bert_output_beauty.zip  -d lora_bert_output_beauty
!unzip lora_bert_output_accs.zip    -d lora_bert_output_accs
!unzip lora_bert_output_bags.zip    -d lora_bert_output_bags



Archive:  lora_bert_output_clothes.zip
   creating: lora_bert_output_clothes/checkpoint-500/
   creating: lora_bert_output_clothes/checkpoint-600/
   creating: lora_bert_output_clothes/runs/
  inflating: lora_bert_output_clothes/adapter_model.safetensors  
  inflating: lora_bert_output_clothes/special_tokens_map.json  
  inflating: lora_bert_output_clothes/adapter_config.json  
  inflating: lora_bert_output_clothes/tokenizer_config.json  
  inflating: lora_bert_output_clothes/README.md  
  inflating: lora_bert_output_clothes/tokenizer.json  
  inflating: lora_bert_output_clothes/sentencepiece.bpe.model  
   creating: lora_bert_output_clothes/runs/Dec19_11-37-33_9a91c2cb5abf/
   creating: lora_bert_output_clothes/runs/Dec19_11-47-48_9a91c2cb5abf/
  inflating: lora_bert_output_clothes/runs/Dec19_11-37-33_9a91c2cb5abf/events.out.tfevents.1766144257.9a91c2cb5abf.357.0  
  inflating: lora_bert_output_clothes/runs/Dec19_11-47-48_9a91c2cb5abf/events.out.tfevents.1766146115.9a91c2cb5abf.357.3 

In [4]:
from transformers import AutoModelForSequenceClassification
from peft import PeftModel
from transformers import AutoTokenizer
import torch

base_model_name = "deepvk/user-bge-m3"  # тот же, что при обучении

In [5]:
bag_tags = ['Удобная', 'Вместительная', 'Компактная', 'Много отделений',
  'Заедает замок', 'Быстро изнашивается', 'Долговечная',
  'Дешево выглядит', 'Высокое качество',
  'Надежный замок', 'Легко чистить','Хорошая упаковка',
  'Есть дефекты', 'Лёгкая', 'Тяжёлая',
  'Прочная ткань', 'Водонепроницаемая',
  'Неприятный запах', 'Соответствует фото', 'Хорошая цена', 'Не соответствует фото',
  'Функциональная', 'Универсальная']

cloth_tags = [
  'Удобный',
  'Приятный материал',
  'Сидит идеально',
  'Идет в размер',
  'Маломерит',
  'Большемерит',
  'Не жмет',
  'Дышит',
  'Качественный принт',
  'Для высокого роста',
  'Плохо пропускает воздух',
  'Просвечивает',
  'Не просвечивает',
  'Не садится после стирки',
  'Садится после стирки',
  'Скатывается ткань',
  'Ткань не скатывается',
  'Качественный',
  'Есть дефекты',
  'Цвет как на фото',
  'Цвет отличается'
]

shoes_tags = [
  'Удобные',
  'Комфортные',
  'Приятная кожа',
  'Приятная замша',
  'Не натирают',
  'Не давят',
  'На среднюю стопу',
  'Маломерят',
  'Большемерки',
  'Требуют разнашивания',
  'Амортизируют',
  'Теплые',
  'Лёгкие',
  'Есть дефекты',
  'Долго носятся',
  'Прочные',
  'Плохо держат ногу',
  'Идут в размер',
  'Не соответствует размеру',
  'Жмут в носке',
  'На широкую стопу',
  'На узкую стопу',
  'Промокают',
  'Не промокают',
  'Недостаточно тёплые',
  'Амортизирующие',
  'Легко чистятся',
  'Сильно натирают'
]

beauty_accs = [
    'Хорошо пенится',
    'Деликатно очищает',
    'Удобный',
    'Отличный состав',
    'Для ночного ухода',
    'Высокая плотность',
    'Питает кожу',
    'Удлиняет ресницы',
    'Сужает поры',
    'Быстро расходуется',
    'Нежный',
    'Легкий аромат',
    'Не оставляет пленки',
    'Освежает лицо',
    'Не стягивает кожу',
    'Слабое отшелушивание',
    'Совместим с кремами',
    'Для сухих волос',
    'Выраженный эффект',
    'Для чувствительной кожи',
    'Не сушит',
    'Освежает дыхание',
    'Отличное качество',
    'Подсушивает',
    'Мягко очищает',
    'Даёт объём',
    'Увлажняет надолго',
]

home_accs_plus_accs = [
    'Плотный',
    'Качественный',
    'Компактный',
    'Быстро расходуется',
    'Удобный',
    'Хорошая впитываемость',
    'Легко стирается',
    'Не деформируется',
    'Плохо просыхающий',
    'Есть дефекты',
    'Не ржавеет',
    'Прочные материалы',
    'Стойкий рисунок',
    'Блекнет после стирки',
    'Приятная ткань',
    'Не протекает',
    'Держит тепло',
    'Долговечный',
    'Дышит',
    'Надежный механизм',
    'Неудобный',
    'Не вызывает аллергии',
    'Маломерит',
    'Большемерит',
    'Образуют катышки',
]

In [6]:
# Принимает теги, имя, путь, отдает модель, токенайзер, теги, имя

def model_tokenizer_tags_name(name, tags, lora_path):
    base_model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=len(tags)
    )

    model_lora = PeftModel.from_pretrained(
        base_model,
        lora_path
    )

    tokenizer_lora = AutoTokenizer.from_pretrained(base_model_name)

    return name, model_lora, tokenizer_lora, tags

In [105]:
models_dict = {
    'Bags': [],
    'Accs_HomeAccs': [],
    'Beauty_Accs': [],
    'Clothes': [],
    'Shoes': []
}


models_dict['Bags'] = model_tokenizer_tags_name('bags', bag_tags, "./lora_bert_output_bags")
models_dict['Accs_HomeAccs'] = model_tokenizer_tags_name('accs', home_accs_plus_accs , "./lora_bert_output_accs")
models_dict['Beauty_Accs'] = model_tokenizer_tags_name('beauty', beauty_accs , "./lora_bert_output_beauty")
models_dict['Clothes'] = model_tokenizer_tags_name('clothes', cloth_tags , "./lora_bert_output_clothes")
models_dict['Shoes'] = model_tokenizer_tags_name('shoes', shoes_tags , "./lora_bert_output_shoes")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/user-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/user-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/user-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
Y

In [106]:
# Получает текст и категорию, выбирает модель, отдает теги

def select_model_and_tags(text, category):

    # Рерут в home accs
    if category not in ('Bags', 'Accs_HomeAccs','Beauty_Accs','Clothes','Shoes'):
        category = 'Accs_HomeAccs'

    # category = 'Bags', например
    name, model, tokenizer, tags = models_dict[category]
    # показ, какой берт был выбран
    print('selected', name)

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.sigmoid(logits)[0]   # (23,)

    threshold = 0.5

    predicted_tags = [
        tags[i] for i, p in enumerate(probs)
        if p >= threshold
    ]

    #print(predicted_tags)
    return predicted_tags

In [39]:
text = "Сумка вместительная и очень удобная"

select_model_and_tags(text, 'Bags')

bags


['Удобная', 'Вместительная']

In [19]:
import pandas as pd

!gdown 1qSdnqNwBZju_rq5CS-Xu5ZnVR11poShH

data = pd.read_csv('/content/lamoda_reviews.csv')
mask = data.groupby('product_sku')['comment_id'].transform('count') >= 5
data = data[mask]
data

Downloading...
From (original): https://drive.google.com/uc?id=1qSdnqNwBZju_rq5CS-Xu5ZnVR11poShH
From (redirected): https://drive.google.com/uc?id=1qSdnqNwBZju_rq5CS-Xu5ZnVR11poShH&confirm=t&uuid=2641eeca-ff64-4091-a818-34265d9814f0
To: /content/lamoda_reviews.csv
100% 313M/313M [00:05<00:00, 54.9MB/s]


,comment_id,product_sku,comment_text,name,good_type,good_subtype
0,395865747,MP002XB078CD,"Купили сыну на день рождения!Теплая, но не жар...",Куртка утепленная,Clothes,OUTWEAR
1,436891792,MP002XB07Z8I,"Приятная вещь, симпатично смотрится",Свитшот,Clothes,SWEATSHIRTS
3,400670943,MP002XC01NLE,Классный комплект,Боди и ползунки,Clothes,TEE-SHIRTS & POLOS
4,388822372,MP002XG03J2N,"Отличные перчатки, плотные, яркие. Ребенку 8 л...",Перчатки,Accs,GLOVES & MITTENS
5,400522318,MP002XG04W4G,Отличные джинсы . Цвет хороший . На худенькую ...,Джинсы,Clothes,JEANS
...,...,...,...,...,...,...
1774262,382001340,RTLADR293401,На человека с животом. Даже s- ка в плечах пор...,Поло,Clothes,KNITWEAR
1774263,386615503,RTLADV038801,Покупала подростку - кроссовки топ по качеству...,Кроссовки,Shoes,SPORT SHOES
1774264,379191175,RTLADV038801,"Очень хорошие кроссовки, цвет бомба!",Кроссовки,Shoes,SPORT SHOES
1774265,380939160,RTLADX703401,"Очень хороший плотный трикотаж. Единственное, ...",Лонгслив,Clothes,TEE-SHIRTS & POLOS


In [31]:
result = (
    data
    .dropna(subset=['comment_text'])
    .groupby('product_sku')
    .agg(
        comment_text=('comment_text', '\n'.join),
        name=('name', 'first'),
        good_type=('good_type', 'first'),
        good_subtype=('good_subtype', 'first'),
        n_comments=('comment_id', 'nunique')
    )
    .reset_index()
)
result

,product_sku,comment_text,name,good_type,good_subtype,n_comments
0,AC001LMDWBR8,"Запах кайф, текстура кайф\nприятно пахнет, увл...",Пена для бритья,Beauty_Accs,SHAVING & DEPILATION,30
1,AC001LUDWBQ4,"Сейчас мой любимый стик. Волшебства не будет, ...",Дезодорант-стик,Beauty_Accs,BODY CARE,45
2,AC001LUDWBQ6,"Хорошая паста, без ядерно-мятного вкуса, ощуще...",Зубная паста,Beauty_Accs,ORAL CARE,55
3,AC001LWJOYE7,Вообще не зашло пахнет лимоном если вы фанат л...,Мыло жидкое,Beauty_Accs,BATH & SHOWER,7
4,AC001LWJOYE9,Отличная щетка. Для тех кто любит пожестче. Че...,Зубная щетка,Beauty_Accs,ORAL CARE,14
...,...,...,...,...,...,...
114060,XD001XW036X6,"очень красиво сидят на ноге, аккуратно\nОни ле...",Кеды,Shoes,SPORT SHOES,8
114061,XD001XW038R4,"Очень удобные, идут в размер, подошва не сколь...",Дутики,Shoes,SNOWBOOTS,5
114062,XD001XW03A94,"На ОГ98. ОБ 99, ОТ78 отлично подошло платье 48...",Платье,Clothes,DRESSES,8
114063,XD001XW03DDG,"Очень клевые!!!\nМягкие,комфортные,стильные!\n...",Кеды,Shoes,SPORT SHOES,5


In [79]:
import numpy as np

# Чтобы не было дисбаланса

# все SKU с Clothes
clothes_skus = result.loc[
    result['good_type'] == 'Clothes',
    'product_sku'
].unique()

# случайно выбираем половину
drop_skus = np.random.choice(
    clothes_skus,
    size=int(len(clothes_skus) // 1.1),
    replace=False
)

# дропаем
result_filtered = result[~result['product_sku'].isin(drop_skus)]

# все SKU с Clothes
clothes_skus = result.loc[
    result['good_type'] == 'Shoes',
    'product_sku'
].unique()

drop_skus = np.random.choice(
    clothes_skus,
    size=int(len(clothes_skus) // 1.5),
    replace=False
)

# дропаем
result_filtered_sh = result_filtered[~result['product_sku'].isin(drop_skus)]

In [87]:
result_filtered_sh['good_type'].value_counts()

,count
good_type,
Shoes,7167
Accs,6772
Clothes,6760
Beauty_Accs,4893
Bags,4235
Home_Accs,2226
Toys,80
Jewellery,8


In [112]:
# Тестовый

sample = result_filtered_sh.sample(n=10)

for _, item in sample.iterrows():

    category = item['good_type']
    text = item['comment_text']
    print(text)

    print('tag:', category)

    tags = select_model_and_tags(text, category)
    print(tags)

    print()


Все понравилось, сын хотел поменьше, из-за этого отказ
Сумка выглядит хорошо, качественно, но когда ее однваешь, как-то она великовата...
Качественная, вместительная, оказалась очень актуальна на отдыхе. Немного объемная, хотелось бы поменьше, но мужа устроили размеры 
качество супер 
Плотная вместительная сумка
Качественно, вместительная, удобная 
tag: Bags
selected bags
['Вместительная', 'Высокое качество']

Яркая, удобная
Кепка топ 
Хорошая кепка
Бейсболка сыну понравилась.
Яркая, натуральный материал, подошла по размеру (нам 4 года).
Покупкой довольна 
Ребенок носит с удовольствием

Отличная кепка.
Суперская кепка 
Яркая,мягкая,в размер 
Советую 
Крутая кепка !! Ребенок доволен 
Пришла мятая, материал как синтетика, хотя состав написано хлопок
Яркая, мальчику на 2 года подошла 
Ребенку понравилась 
Прослужила уже долго и прошла испытание походом, до сих пор в строю
Хорошая кепка , для мальчика 4 года подошла идеально. Размер соответствует)
Прикольная, за такие деньги супер
Качестве